In [1]:
from pathlib import Path
from typing import List, Dict

from expert_system.student import StudentExpertSystem
from loaders.data import DataLoader
from loaders.fact import FactLoader
from loaders.rule import RuleLoader

In [2]:
facts_schema = FactLoader.load(Path('examples/facts.json'))
data = DataLoader.load(
    Path('examples\students.json')
)
rules = RuleLoader.load(Path('examples/rules.json'))

In [18]:
facts_schema.facts

{'assignments_completed': FactDefinition(name='assignments_completed', type=<FactType.NUMBER: 'number'>, description='Количество сданных заданий по курсу, в процентах', required=True, default=None, min_value=0, max_value=100, enum_values=None, unit='%'),
 'report_completion': FactDefinition(name='report_completion', type=<FactType.NUMBER: 'number'>, description='Процент выполнения отчета по предмету', required=True, default=None, min_value=0, max_value=100, enum_values=None, unit='%'),
 'theory_test_score': FactDefinition(name='theory_test_score', type=<FactType.NUMBER: 'number'>, description='Результат теста по теоретическому курсу', required=True, default=None, min_value=0, max_value=100, enum_values=None, unit='баллы'),
 'attendance': FactDefinition(name='attendance', type=<FactType.NUMBER: 'number'>, description='Посещаемость занятий', required=False, default=100, min_value=0, max_value=100, enum_values=None, unit='%'),
 'student_id': FactDefinition(name='student_id', type=<FactTyp

In [3]:
data

[{'student_id': 'STU001',
  'name': 'Иван Петров',
  'assignments_completed': 95,
  'report_completion': 92,
  'theory_test_score': 88,
  'attendance': 95},
 {'student_id': 'STU002',
  'name': 'Мария Сидорова',
  'assignments_completed': 75,
  'report_completion': 80,
  'theory_test_score': 70,
  'attendance': 85},
 {'student_id': 'STU003',
  'name': 'Алексей Козлов',
  'assignments_completed': 55,
  'report_completion': 60,
  'theory_test_score': 58,
  'attendance': 75},
 {'student_id': 'STU004',
  'name': 'Екатерина Новикова',
  'assignments_completed': 30,
  'report_completion': 40,
  'theory_test_score': 45,
  'attendance': 60},
 {'student_id': 'STU005',
  'name': 'Дмитрий Волков',
  'assignments_completed': 85,
  'report_completion': 88,
  'theory_test_score': 78,
  'attendance': 92}]

In [4]:
rules

[Rule(name='Отличная подготовка', conditions=[Condition(fact_name='assignments_completed', operator='gte', value=90), Condition(fact_name='report_completion', operator='gte', value=90), Condition(fact_name='theory_test_score', operator='gte', value=85)], actions=[Action(action_type='set_result', parameters={'probability': 98, 'status': 'Отличная подготовка', 'recommendation': 'Вы полностью готовы к экзамену! Продолжайте в том же духе.'})], priority=10, description='Все показатели на отличном уровне'),
 Rule(name='Очень хорошая подготовка', conditions=[Condition(fact_name='assignments_completed', operator='between', value=[80, 90]), Condition(fact_name='report_completion', operator='gte', value=85), Condition(fact_name='theory_test_score', operator='gte', value=75)], actions=[Action(action_type='set_result', parameters={'probability': 92, 'status': 'Очень хорошая подготовка', 'recommendation': 'Отличные результаты! Рекомендуется завершить оставшиеся задания.'})], priority=9, description

In [5]:
def format_results(
        student_name: str,
        student_id: str,
        facts: Dict,
        results: Dict,
        fired_rules: List[str]
):
    """Красивое форматирование результатов."""
    print("=" * 80)
    print(f"РЕЗУЛЬТАТЫ ОЦЕНКИ СТУДЕНТА")
    print("=" * 80)
    print(f"\nСтудент: {student_name}")
    print(f"ID: {student_id}")
    print("\n" + "-" * 80)
    print("ВХОДНЫЕ ДАННЫЕ:")
    print("-" * 80)
    print(f"  • Сданные задания: {facts['assignments_completed']}%")
    print(f"  • Выполнение отчета: {facts['report_completion']}%")
    print(f"  • Тест по теории: {facts['theory_test_score']} баллов")
    print(f"  • Посещаемость: {facts.get('attendance', 100)}%")

    print("\n" + "-" * 80)
    print("ВЫХОДНЫЕ ДАННЫЕ:")
    print("-" * 80)
    print(f"\n📊 СТАТУС: {results['status']}")
    print(f"📈 ВЕРОЯТНОСТЬ ДОПУСКА К СЕССИИ: {results['admission_probability']}%")

    print("\n💡 РЕКОМЕНДАЦИИ:")
    for i, rec in enumerate(results['recommendations'], 1):
        print(f"   {i}. {rec}")

    print("\n" + "-" * 80)
    print(f"Сработавшие правила: {', '.join(fired_rules)}")
    print("=" * 80)
    print()


In [6]:
system = StudentExpertSystem()
system.add_rules(rules)

In [7]:
all_results = []

In [8]:
for student_info in data:
    system = StudentExpertSystem()
    system.add_rules(rules)
    system.add_facts(student_info)

    system.run_evaluation()
    results = system.get_results()

    format_results(
            student_info['name'],
            student_info['student_id'],
            system.get_facts(),
            results,
            system.fired_rules
        )

РЕЗУЛЬТАТЫ ОЦЕНКИ СТУДЕНТА

Студент: Иван Петров
ID: STU001

--------------------------------------------------------------------------------
ВХОДНЫЕ ДАННЫЕ:
--------------------------------------------------------------------------------
  • Сданные задания: 95%
  • Выполнение отчета: 92%
  • Тест по теории: 88 баллов
  • Посещаемость: 95%

--------------------------------------------------------------------------------
ВЫХОДНЫЕ ДАННЫЕ:
--------------------------------------------------------------------------------

📊 СТАТУС: Отличная подготовка
📈 ВЕРОЯТНОСТЬ ДОПУСКА К СЕССИИ: 100%

💡 РЕКОМЕНДАЦИИ:
   1. Вы полностью готовы к экзамену! Продолжайте в том же духе.
   2. Отличная посещаемость! Это увеличивает ваши шансы на допуск.

--------------------------------------------------------------------------------
Сработавшие правила: Отличная подготовка, Бонус за посещаемость

РЕЗУЛЬТАТЫ ОЦЕНКИ СТУДЕНТА

Студент: Мария Сидорова
ID: STU002

-------------------------------------------------